In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
import time
df = spark.createDataFrame([
    ('id_cliente-1',  'cat-1, cat-2, cat-3'),
    ('id_cliente-2',  'cat-1, cat-4, cat-5'),
    ('id_cliente-3',  'cat-6, cat-7'),
    ('id_cliente-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('id_cliente-5',  'cat-8, cat-10'),
    ('id_cliente-6',  'cat-1, cat-9, cat-10'),
    ('id_cliente-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('id_cliente-8',  'cat-7, cat-9'),
    ('id_cliente-9',  'cat-1'),
    ('id_cliente-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

df2 = spark.createDataFrame([
    ('id_cliente-1',  'cat-1, cat-2, cat-3, cat-15'),
    ('id_cliente-2',  'cat-1, cat-4, cat-5, cat-11, cat-14'),
    ('id_cliente-3',  'cat-4, cat-14, cat-15'),
    ('id_cliente-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('id_cliente-5',  'cat-8, cat-10, cat-11'),
    ('id_cliente-6',  'cat-1, cat-9, cat-10, cat-11, cat-13'),
    ('id_cliente-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('id_cliente-8',  'cat-7, cat-9, cat-12, cat-13, cat-14'),
    ('id_cliente-9',  'cat-2'),
    ('id_cliente-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

df2.cache()
df.cache()
from pyspark.sql.types import Row

In [2]:
def FD1(df1):
  # Seu script

  def createColumns(line, valueMax):

      newLine = {}
      newLine['client-id'] = line['id_cliente']

      for i in range(1, valueMax + 1):
          if str(i) in str(line['categorias']):
              newLine['cat-' + str(i)] = 1
          else:
              newLine['cat-' + str(i)] = 0

      return Row(**newLine)


  result2 = (df
             .withColumn("cat", explode(split("categorias", ",")))
             .withColumn("t", (split("cat", "-")[1]).cast("int"))
            )
  valueMax =  result2.select(max(result2["t"])).take(1)[0][0]
  df3 = df.rdd.map(lambda line: createColumns(line, valueMax)).toDF()


  lstSelect = []
  lstSelect.append('client-id')

  for i in range(1, valueMax+1):
      lstSelect.append("cat-" + str(i))

  df3.select([field_name for field_name in lstSelect]).show()


In [3]:
def FD2(df2):
  
  # Seu script
  def createColumns(line, valueMax):

      newLine = {}
      newLine['client-id'] = line['id_cliente']

      for i in range(1, valueMax + 1):
          if str(i) in str(line['categorias']):
              newLine['cat-' + str(i)] = 1
          else:
              newLine['cat-' + str(i)] = 0

      return Row(**newLine)


  result2 = (df2
             .withColumn("cat", explode(split("categorias", ",")))
             .withColumn("t", (split("cat", "-")[1]).cast("int"))
            )
  valueMax =  result2.select(max(result2["t"])).take(1)[0][0]
  dfx = df2.rdd.map(lambda line: createColumns(line, valueMax)).toDF()



  lstSelect = []
  lstSelect.append('client-id')

  for i in range(1, valueMax+1):
      lstSelect.append("cat-" + str(i))

  dfx.select([field_name for field_name in lstSelect]).show()

In [4]:
import threading

In [5]:
start = time.time()

T1 = threading.Thread(target=FD1, args=(df,))
T2 = threading.Thread(target=FD2, args=(df2,))

# Iniciando execução das threads.
# Starting execution of threads.
T1.start()
T2.start()

# Pausando execução das threads para seguir o fluxo principal. Sem eles, a thread rodará em paralelo.
# Pausing thread execution to follow main stream. Without them, the thread will run in parallel.
T1.join()
T2.join()

print(time.time() - start)

+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
 client-id|cat-1|cat-2|cat-3|cat-4|cat-5|cat-6|cat-7|cat-8|cat-9|cat-10|cat-11|cat-12|cat-13|cat-14|cat-15|
+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
 id_cliente-1| 1| 1| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1|
 id_cliente-2| 1| 0| 0| 1| 1| 0| 0| 0| 0| 0| 1| 0| 0| 1| 0|
 id_cliente-3| 1| 0| 0| 1| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 1|
 id_cliente-4| 1| 1| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0|
 id_cliente-5| 1| 0| 0| 0| 0| 0| 0| 1| 0| 1| 1| 0| 0| 0| 0|
 id_cliente-6| 1| 0| 1| 0| 0| 0| 0| 0| 1| 1| 1| 0| 1| 0| 0|
 id_cliente-7| 1| 0| 0| 1| 1| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0|
 id_cliente-8| 1| 1| 1| 1| 0| 0| 1| 0| 1| 0| 0| 1| 1| 1| 0|
 id_cliente-9| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
id_cliente-10| 1| 1| 1| 1| 1| 1| 1| 1| 0| 1| 0| 0| 0| 0| 0|
+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+

+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
 client-id|cat-1|cat-2|cat-3|cat-4|cat-5|cat-6|cat-7|cat-8|cat-9|cat-10|
+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
 id_cliente-1| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0|
 id_cliente-2| 1| 0| 0| 1| 1| 0| 0| 0| 0| 0|
 id_cliente-3| 0| 0| 0| 0| 0| 1| 1| 0| 0| 0|
 id_cliente-4| 1| 1| 0| 0| 0| 0| 1| 0| 0| 1|
 id_cliente-5| 1| 0| 0| 0| 0| 0| 0| 1| 0| 1|
 id_cliente-6| 1| 0| 0| 0| 0| 0| 0| 0| 1| 1|
 id_cliente-7| 1| 0| 0| 1| 1| 0| 0| 0| 0| 1|
 id_cliente-8| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0|
 id_cliente-9| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
id_cliente-10| 1| 1| 1| 1| 1| 1| 1| 1| 0| 1|
+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+

0.772026062012